In [ ]:
%%capture
!pip install qiskit qiskit_ibm_runtime qiskit-aer matplotlib

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from math import gcd, ceil, log2
from fractions import Fraction
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit.circuit.library import QFT
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Session
from qiskit.visualization import plot_histogram

In [ ]:
N = 77
n = ceil(log2(N))
q = QuantumRegister(2*n, "q")
t = QuantumRegister(n, "t")
c = ClassicalRegister(2*n, "c")
state = False
max_tentativas = 20
tentativas = 0

In [ ]:
# backend = FakeKyiv()
service = QiskitRuntimeService(channel="ibm_quantum", token = "")
backend = service.least_busy(operational=True, simulator=False)
if backend is None:
  raise ValueError("Nenhum backend disponível. Verifique sua conexão com a IBM Quantum.")
else:
  print("backend selecionado:", backend)

In [ ]:
def Uf(a, circ):
    """
    Uf: simula a operação de exponenciação modular controlada usada no algoritmo de Shor.
    Para cada qubit de controle i, aplica uma rotação condicional de fase proporcional a a^(2^i) mod N.

    Parâmetros:
    - a: base da exponenciação modular (a^x mod N)
    - circ: circuito quântico (QuantumCircuit) que contém os registradores q (controle) e t (target)

    Notas:
    - Este é um modelo simplificado que usa CP (Controlled-Phase) para representar o efeito
      da unidade modular U_f: |x⟩|1⟩ → |x⟩|a^x mod N⟩ através de fases.
    - A operação não é reversível no sentido estrito de Uf real, mas serve como aproximação para simulação.
    """

    # Para cada qubit de controle i no registrador de fase (q), simulamos Uf aplicando uma rotação de fase controlada
    for i in range(2 * n):
        # Calcula a^(2^i) mod N, que representa a contribuição de cada bit de x no expoente
        mod_exp = pow(a, 2**i, N)

        # Aplica uma rotação de fase condicional (Controlled-Phase) proporcional ao resultado da mod_exp
        # Isso simula o efeito da função a^x mod N em fase
        circ.cp(2 * np.pi * mod_exp / N, q[i], t)

In [ ]:
def retrieve_denominator(measured_decimal, n, N):
    """
    Estima o período r a partir da medição 'measured_decimal', onde:
    - n: número de qubits de contagem
    - N: número a ser fatorado (limite para denominador)

    Retorna:
    - O denominador da melhor fração contínua de x/2^n com denominador ≤ N
    """
    phase = measured_decimal / (2 ** n)
    frac = Fraction(phase).limit_denominator(N)
    return frac.denominator

In [ ]:
def etapa_2_verificacao_preliminar(N, limites_primos=[2, 3, 5, 7, 11, 13, 17, 19]):
    """
    Etapa 2 do algoritmo de Shor: verificação clássica de fatores triviais.

    Parâmetro:
    - N: número inteiro composto a ser fatorado
    - limites_primos: primos pequenos para tentativa de divisão

    Retorna:
    - (True, fatores) se algum fator trivial for encontrado
    - (False, None) se não houver fatoração trivial
    """
    if N % 2 == 0:
        return True, (2, N // 2)

    for p in limites_primos:
        if N % p == 0:
            return True, (p, N // p)

    return False, None

In [ ]:
def etapa3(N):
    """
    Etapa 3 do algoritmo de Shor: escolha aleatória de um número 'a'
    tal que 1 < a < N e gcd(a, N) = 1.

    Parâmetro:
    - N: número inteiro composto a ser fatorado

    Retorna:
    - a: base escolhida para uso na etapa quântica do algoritmo (a^x mod N)
    """

    # Sorteia um número a entre 2 e N-1
    a = random.randint(2, N - 1)

    # Repete enquanto a não for coprimo de N (ou seja, enquanto gcd(a, N) > 1)
    # Se gcd(a, N) > 1, então já é possível retornar um fator de N
    while gcd(a, N) > 1:
        a = random.randint(2, N - 1)

    # Exibe a base escolhida (útil para depuração)
    print(f"\n🎲 Tentativa {tentativas}: base escolhida a = {a}")

    # Retorna o valor de a para as próximas etapas
    return a
# a = etapa3(N)

In [ ]:
def etapa4(q, t, c, n, backend, a):
    """
    Etapa 4 do algoritmo de Shor: cálculo do período r via estimativa de fase quântica (QPE).
    Utiliza a função f(x) = a^x mod N codificada como Uf.

    Parâmetros:
    - q: registrador de contagem (QuantumRegister com 2n qubits)
    - t: registrador alvo (QuantumRegister com 1 qubit)
    - c: registrador clássico (ClassicalRegister com 2n bits)
    - n: número de qubits necessários para representar N (n = ceil(log2(N)))
    - backend: backend de execução (ex: Aer simulator)
    - a: base usada na exponenciação modular (a^x mod N)

    Retorna:
    - sorted_measurements: lista ordenada de medições decrescentes [(bitstring, contagem)]
    """

    # Cria circuito com registradores de contagem, alvo e medição
    circuito = QuantumCircuit(q, t, c)

    # Inicializa o registrador alvo (target) no estado |1⟩
    circuito.x(t)

    # Barreira para separar visualmente preparação da QPE
    circuito.barrier()

    # Aplica Hadamard nos qubits de contagem para criar superposição uniforme
    circuito.h(q)

    # Aplica a operação modular controlada Uf(a)
    # Esta operação representa a função f(x) = a^x mod N
    # Deve ser definida externamente e adicionada como circuito reversível
    Uf(a, circuito)

    # Aplica a transformada quântica de Fourier inversa (QFT†) nos qubits de contagem
    circuito.append(QFT(2 * n, inverse=True), q)

    # Mede os qubits de contagem no registrador clássico
    circuito.measure(q, c)

    # Transpila o circuito para o backend alvo
    transpiled = transpile(circuito, backend)


    # Executa o circuito no backend com 1024 shots
    sampler = Sampler(backend)
    job = sampler.run([transpiled])
    pub_result = job.result()[0]

    # Obtém os resultados de medição
    counts = pub_result.data.c.get_counts()
    plot_histogram(counts)

    # Ordena os resultados por frequência (do mais frequente ao menos)
    sorted_measurements = sorted(counts.items(), key=lambda x: x[1], reverse=True)

    # Retorna os resultados ordenados
    return sorted_measurements
# sorted_measurements = etapa4(q, t, c, n, backend)

In [ ]:
def etapa5(x, N):
    """
    Etapa 5 do algoritmo de Shor: verificação do valor do período r estimado.

    A ideia é verificar se a^(r/2) mod N resulta em um valor que torna impossível extrair os fatores.
    Isso acontece quando:
        - a^(r/2) ≡ 1 mod N, ou
        - a^(r/2) ≡ -1 mod N

    Parâmetros:
    - x: valor de a^(r/2) mod N
    - N: número que está sendo fatorado

    Retorna:
    - True se o período é inválido (isto é, x ≡ ±1 mod N)
    - None (implícito) se o período pode ser útil (não é tratado aqui, apenas detecta casos inválidos)
    """

    # Se a^(r/2) ≡ 1 ou ≡ -1 (mod N), o período r não é útil
    if x == 1 or x == N - 1:
        return True  # indica que o período falha a verificação
# if etapa5(x, N) == True:
#   continue

In [ ]:
def etapa6(x, N):
    """
    Etapa 6 do algoritmo de Shor: cálculo dos fatores não triviais de N.

    Após encontrar um período r válido, calcula-se:
        x = a^(r/2) mod N
    Em seguida, utiliza-se:
        p = gcd(x - 1, N)
        q = gcd(x + 1, N)

    Parâmetros:
    - x: valor de a^(r/2) mod N (deve ter passado pela verificação da etapa 5)
    - N: número inteiro composto a ser fatorado

    Retorna:
    - (p, q): tupla contendo os fatores candidatos de N
    """

    # Calcula o mdc entre (x - 1) e N → possível fator de N
    p = gcd(x - 1, N)

    # Calcula o mdc entre (x + 1) e N → outro possível fator de N
    q_ = gcd(x + 1, N)

    # Retorna ambos os fatores candidatos
    return p, q_
# p, q_ = etapa6(x, N)

In [ ]:
if etapa_2_verificacao_preliminar(N, limites_primos = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41,
                      43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97])[0]:
    print(f"\n🎯 Resultado do Algoritmo de Shor:{N}\n" + "-"*40)
    while not state and tentativas < max_tentativas:
        tentativas += 1

        # Etapa 3 – escolha aleatória de a tal que gcd(a, N) = 1
        a = etapa3(N)

        # Etapa 4 – preparar circuito QPE
        sorted_measurements = etapa4(q, t, c, n, backend, a)

        # Etapas 5 e 6 – Verificação do período e cálculo dos fatores
        for measurement, _ in sorted_measurements:
            decimal_value = int(measurement, 2)
            r = retrieve_denominator(decimal_value, n, N)

            if r % 2 == 0:
                x = pow(a, r // 2, N)

                # Etapa 5: ignorar se x ≡ ±1 mod N
                if etapa5(x, N) == True:
                  continue

                # Etapa 6: cálculo dos fatores
                p, q_ = etapa6(x, N)

                if 1 < p < N and 1 < q_ < N and p * q_ == N:
                    print(f"✅ Fatores encontrados: {p}, {q_}")
                    state = True
                    break